In [13]:
import io
import pickle5 as pickle

In [31]:
dir_path = './amazon/'

In [27]:
def preprocess_words(words: list) -> str:
    document = ''
    for i in range(len(words)):
        tmp = words[i].split(':')
        for j in range(int(tmp[1])):
            document += tmp[0] + ' '
    return document

In [26]:
def file_to_corpus(file: io.TextIOWrapper) -> (list, list):
    corpus = []
    labels = []
    for review in file:
        contents = review.split()
        words = contents[:-1]
        document = preprocess_words(words)
        label = contents[-1].split(':')[1]
        y = 0 if label == 'negative' else 1
        corpus.append(document)
        labels.append(y)
    return corpus, labels

In [42]:
domains = ['books', 'dvd', 'electronics', 'kitchen']
kinds = ['negative', 'positive', 'unlabeled']
for domain in domains:
    save_path = './amazon/%s/%s.pkl' % (domain, domain)
    corpus_train = []
    corpus_test = []
    labels_train = []
    labels_test = []
    # training set (labeled)
    for kind in kinds[:-1]:
        file_path = './amazon/%s/%s.review' % (domain, kind)
        with open(file_path, 'r') as r:
            corpus, labels = file_to_corpus(r)
            corpus_train.extend(corpus)
            labels_train.extend(labels)
    # test set (unlabeled)
    file_path = './amazon/%s/%s.review' % (domain, kinds[-1])
    with open(file_path, 'r') as r:
        corpus, labels = file_to_corpus(r)
        corpus_test.extend(corpus)
        labels_test.extend(labels)
    with open(save_path, 'wb') as w:
        pickle.dump([corpus_train, labels_train, corpus_test, labels_test], w)

In [44]:
for domain in domains:
    save_path = './amazon/%s/%s.pkl' % (domain, domain)
    with open(save_path, 'rb') as r:
        corpus_train, labels_train, corpus_test, labels_test = pickle.load(r)
    print(domain + ': ' + str(len(labels_train)))
    print(domain + ': ' + str(len(labels_test)))

books: 2000
books: 4465
dvd: 2000
dvd: 3586
electronics: 2000
electronics: 5681
kitchen: 2000
kitchen: 5945
